In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!apt-get update
!apt-get install -y chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 https://cli.github.com/packages stable InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [80.2 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,961 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,232 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Pac

In [ ]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

# selenium 임포트 및 셋업
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time
from selenium.webdriver.common.by import By

chrome_options = Options()
chrome_options.add_argument("--headless") # GUI 없는 환경
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# 드라이버 실행
driver = webdriver.Chrome(options=chrome_options)


In [ ]:
import os

def save_song_info_to_drive(singer, title, album, release_date, genre, like_count, lyrics):
    # 구글 드라이브 내 기본 저장 경로
    base_path = "/content/drive/MyDrive/graduateproject/music_data"

    # 폴더명과 파일명: 공백과 특수문자 제거
    safe_singer = singer.replace(" ", "_").replace("/", "-")
    safe_title = title.replace(" ", "_").replace("/", "-")
    folder_name = f"{safe_singer}_{safe_title}"

    # 최종 폴더 경로 생성
    folder_path = os.path.join(base_path, folder_name)
    os.makedirs(folder_path, exist_ok=True)

    # 파일 경로 설정
    filename = f"info_{safe_title}_{safe_singer}.txt"
    file_path = os.path.join(folder_path, filename)

    # 텍스트 저장
    with open(file_path, "w", encoding="utf-8") as f:
        f.write(f"곡 제목: {title}\n")
        f.write(f"가수: {singer}\n")
        f.write(f"앨범: {album}\n")
        f.write(f"발매일: {release_date}\n")
        f.write(f"장르: {genre}\n")
        f.write(f"좋아요 수: {like_count}\n")
        f.write("\n[가사]\n")
        f.write(lyrics)

    print(f"✅ 저장 완료: {file_path}")


In [4]:
import os
import time
import re
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

# 크롤링된 곡 정보들을 저장할 리스트
all_songs = []

# 멜론 메인 접속
driver.get("https://www.melon.com/")
time.sleep(3)

# 검색창에 "아이유" 입력
search_input = driver.find_element(By.ID, "top_search")
search_input.send_keys("아이유")
search_input.send_keys(Keys.ENTER)
time.sleep(3)

# 곡 탭 클릭
song_tab = driver.find_element(By.XPATH, '//a[@title="곡 - 페이지 이동"]')
song_tab.click()
time.sleep(3)

# 곡 리스트 추출
songs = driver.find_elements(By.CSS_SELECTOR, "div.tb_list.d_song_list.songTypeOne table tbody tr")
print(f"총 곡 수: {len(songs)}")

# 상위 N개 곡 반복 (여기서는 5개만)
for i, song in enumerate(songs[:5]):
    try:
        title_elem = song.find_element(By.CSS_SELECTOR, "a.fc_gray")
        title = title_elem.get_attribute("title")

        detail_link_elem = song.find_element(By.CSS_SELECTOR, "a.btn_icon_detail")
        onclick = detail_link_elem.get_attribute("href")
        match = re.search(r"goSongDetail\('(\d+)'\)", onclick)

        if match:
            song_id = match.group(1)
            url = f"https://www.melon.com/song/detail.htm?songId={song_id}"
            driver.get(url)
            time.sleep(2)

            try:
                title = driver.find_element(By.CSS_SELECTOR, "div.song_name").text.strip()
                singer = driver.find_element(By.CSS_SELECTOR, "div.artist a.artist_name").text.strip()
                album = driver.find_element(By.CSS_SELECTOR, "div.meta dl.list > dd:nth-of-type(1)").text.strip()
                release_date = driver.find_element(By.CSS_SELECTOR, "div.meta dl.list > dd:nth-of-type(2)").text.strip()
                genre = driver.find_element(By.CSS_SELECTOR, "div.meta dl.list > dd:nth-of-type(3)").text.strip()
                like_count = driver.find_element(By.CSS_SELECTOR, "span#d_like_count").text.strip().replace(",", "")
                lyrics = driver.find_element(By.ID, "d_video_summary").text.strip()

                print(f"🎵 {i+1}. {title} 저장 중...")

                # 딕셔너리로 저장
                all_songs.append({
                    "title": title,
                    "singer": singer,
                    "album": album,
                    "release_date": release_date,
                    "genre": genre,
                    "like_count": like_count,
                    "lyrics": lyrics
                })

            except Exception as e:
                print(f"❌ 상세 정보 추출 실패: {e}")

            driver.back()
            time.sleep(2)
            songs = driver.find_elements(By.CSS_SELECTOR, "div.tb_list.d_song_list.songTypeOne table tbody tr")
        else:
            print(f"{i+1}. {title} → ❌ URL 추출 실패")
    except Exception as e:
        print(f"{i+1}. ❌ 에러 발생: {e}")

# 🔽 크롤링 완료 후 CSV 저장
df = pd.DataFrame(all_songs)
save_path = "/content/drive/MyDrive/graduateproject/music_lyric/songs.csv"

# 기존에 있는 csv유지하려면 이렇게 해야함
# --> 이 코드 수해서 가수 이름마다 열고 닫기 하지 말고 가수 여러명 한번에 입력하고 한방에 다 쓰고 한방에 닫는 코드로 바꿔야함
#df.to_csv(save_path, mode='a', index=False, encoding="utf-8-sig", header=not os.path.exists(save_path))

df.to_csv(save_path, index=False, encoding="utf-8-sig")
print(f"\n✅ CSV 저장 완료: {save_path}")


NameError: name 'driver' is not defined

In [ ]:
!pip install yt_dlp
!apt-get install ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Cookie.txt to Cookie.txt


In [ ]:
import shutil
shutil.copy("/content/Cookie.txt", "/content/cookies.txt")

'/content/cookies.txt'

In [ ]:
def download_youtube_mp3(query, output_dir="downloads"):
    driver.get("https://www.youtube.com/")
    time.sleep(3)

    search_box = driver.find_element(By.NAME, "search_query")
    search_box.send_keys(query)
    search_box.send_keys(Keys.RETURN)
    time.sleep(4)

    # 여러 영상 중 첫 번째 href 있는 영상 찾기
    videos = driver.find_elements(By.ID, "video-title")
    video_url = None
    for video in videos:
        video_url = video.get_attribute("href")
        if video_url:
            break

    if not video_url:
        raise ValueError("❌ 영상 URL을 찾지 못했습니다.")

    print(f"🎬 영상 URL: {video_url}")

    # 다운로드
    os.makedirs(output_dir, exist_ok=True)
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': os.path.join(output_dir, f'{query}.%(ext)s'),
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '128',
        }],
        'quiet': True,
        'noplaylist': True,
        'cookiefile': '/content/cookies.txt'
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([video_url])

    print(f"✅ 다운로드 완료: {query}.mp3")


In [ ]:
download_youtube_mp3("jazzyfact vibra", output_dir="/content/drive/MyDrive/graduateproject/music_data/mp3s")

🎬 영상 URL: https://www.youtube.com/watch?v=kwQAn31d7gg&pp=ygUPamF6enlmYWN0IHZpYnJh0gcJCYsJAYcqIYzv
✅ 다운로드 완료: jazzyfact vibra.mp3


In [ ]:
import re
import time
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [ ]:
def crawl_song_info(driver, artist_name, max_count=5):


    all_songs = []

    # 멜론 메인 접속
    driver.get("https://www.melon.com/")
    time.sleep(3)

    # 검색
    search_input = driver.find_element(By.ID, "top_search")
    search_input.send_keys(artist_name)
    search_input.send_keys(Keys.ENTER)
    time.sleep(3)

    # '곡' 탭 클릭
    song_tab = driver.find_element(By.XPATH, '//a[@title="곡 - 페이지 이동"]')
    song_tab.click()
    time.sleep(3)

    # 곡 리스트 추출
    songs = driver.find_elements(By.CSS_SELECTOR, "div.tb_list.d_song_list.songTypeOne table tbody tr")
    print(f"총 곡 수: {len(songs)}")

    for i, song in enumerate(songs[:max_count]):
        try:
            title_elem = song.find_element(By.CSS_SELECTOR, "a.fc_gray")
            title = title_elem.get_attribute("title")

            detail_link_elem = song.find_element(By.CSS_SELECTOR, "a.btn_icon_detail")
            onclick = detail_link_elem.get_attribute("href")
            match = re.search(r"goSongDetail\('(\d+)'\)", onclick)

            if match:
                song_id = match.group(1)
                url = f"https://www.melon.com/song/detail.htm?songId={song_id}"
                driver.get(url)
                time.sleep(2)

                try:
                    album_img_url = driver.find_element(By.CSS_SELECTOR, "div.thumb#d_song_org img").get_attribute("src")
                    title = driver.find_element(By.CSS_SELECTOR, "div.song_name").text.strip()
                    singer = driver.find_element(By.CSS_SELECTOR, "div.artist a.artist_name").text.strip()
                    album = driver.find_element(By.CSS_SELECTOR, "div.meta dl.list > dd:nth-of-type(1)").text.strip()
                    release_date = driver.find_element(By.CSS_SELECTOR, "div.meta dl.list > dd:nth-of-type(2)").text.strip()
                    genre = driver.find_element(By.CSS_SELECTOR, "div.meta dl.list > dd:nth-of-type(3)").text.strip()
                    like_count = driver.find_element(By.CSS_SELECTOR, "span#d_like_count").text.strip().replace(",", "")
                    lyrics = driver.find_element(By.ID, "d_video_summary").text.strip()

                    print(f"🎵 {i+1}. {title} 정보 저장 중...")

                    all_songs.append({
                        "title": title,
                        "singer": singer,
                        "album": album,
                        "release_date": release_date,
                        "genre": genre,
                        "like_count": like_count,
                        "lyrics": lyrics,
                        "album_img_url": album_img_url  # 나중에 별도로 이미지 저장
                    })

                except Exception as e:
                    print(f"❌ 상세 정보 추출 실패: {e}")

                driver.back()
                time.sleep(2)
                songs = driver.find_elements(By.CSS_SELECTOR, "div.tb_list.d_song_list.songTypeOne table tbody tr")

        except Exception as e:
            print(f"{i+1}. ❌ 에러 발생: {e}")

    return all_songs


In [ ]:
def save_album_image(title, album, album_img_url, save_dir):
    import os
    import re
    import requests

    os.makedirs(save_dir, exist_ok=True)

    try:
        # 해상도 업
        album_img_url = album_img_url.replace("/resize/282", "/resize/1000")

        # 파일명 안전화
        safe_album = re.sub(r'[\\/:*?"<>|]', '_', album)
        safe_title = re.sub(r'[\\/:*?"<>|]', '_', title)
        filename = f"{safe_album}_{safe_title}_cover.jpg"
        img_path = os.path.join(save_dir, filename)

        # 이미지 다운로드
        img_data = requests.get(album_img_url).content
        with open(img_path, "wb") as f:
            f.write(img_data)

        print(f"🖼️ 앨범 이미지 저장 완료: {filename}")
        return filename

    except Exception as e:
        print(f"❌ 이미지 저장 실패 ({title}): {e}")
        return None


In [ ]:
import os

# 파일 존재 여부 확인
file_exists = os.path.exists(csv_path)

artist = "빈지노"
save_dir = "/content/drive/MyDrive/graduateproject/music_lyric/album_images"
csv_path = "/content/drive/MyDrive/graduateproject/music_lyric/songs.csv"

songs = crawl_song_info(driver, artist, max_count=5)

# 이미지 저장 및 파일명 추가
for song in songs:
    img_file = save_album_image(song["title"], song["album"], song["album_img_url"], save_dir)
    song["image_filename"] = img_file  # CSV 기록용

# CSV 저장
df = pd.DataFrame(songs)
df.to_csv(csv_path, mode='a', index=False, encoding="utf-8-sig",header=not file_exists)
print(f"\n✅ CSV 저장 완료: {csv_path}")

총 곡 수: 50
🎵 1. Aqua Man 정보 저장 중...
🎵 2. Boogie On & On 정보 저장 중...
🎵 3. Always Awake (Bonus Track) 정보 저장 중...
🎵 4. Nike Shoes (Feat. Dynamic Duo) 정보 저장 중...
🎵 5. Break 정보 저장 중...
🖼️ 앨범 이미지 저장 완료: 2 4 _ 2 6_Aqua Man_cover.jpg
🖼️ 앨범 이미지 저장 완료: 2 4 _ 2 6_Boogie On & On_cover.jpg
🖼️ 앨범 이미지 저장 완료: 24_26 (5th Anniversary Remaster Edition)_Always Awake (Bonus Track)_cover.jpg
🖼️ 앨범 이미지 저장 완료: 2 4 _ 2 6_Nike Shoes (Feat. Dynamic Duo)_cover.jpg
🖼️ 앨범 이미지 저장 완료: Break_Break_cover.jpg

✅ CSV 저장 완료: /content/drive/MyDrive/graduateproject/music_lyric/songs.csv


In [ ]:
import pandas as pd
import yt_dlp
import os
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

# ✅ CSV 파일 불러오기
csv_path = "/content/drive/MyDrive/graduateproject/music_lyric/songs.csv"
df = pd.read_csv(csv_path)

# ✅ 유튜브 드라이버 설정 (이미 열려 있다면 생략)
driver.get("https://www.youtube.com/")
time.sleep(2)

# ✅ 다운로드 함수 정의
def download_mp3_from_youtube(query, output_dir="downloads"):
    driver.get("https://www.youtube.com/")
    time.sleep(2)

    # 검색
    search_box = driver.find_element(By.NAME, "search_query")
    search_box.clear()
    search_box.send_keys(query)
    search_box.send_keys(Keys.RETURN)
    time.sleep(3)

    # 첫 번째 영상 URL 가져오기
    try:
        video = driver.find_element(By.ID, "video-title")
        video_url = video.get_attribute("href")
        print(f"🎬 영상: {query} → {video_url}")
    except:
        print(f"❌ 검색 실패: {query}")
        return

    # MP3 다운로드 설정
    os.makedirs(output_dir, exist_ok=True)
    ydl_opts = {
        'format': 'bestaudio[ext=m4a]/bestaudio/best',
        'outtmpl': os.path.join(output_dir, f'{query}.%(ext)s'),
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '128',
        }],
        'quiet': True,
        'noplaylist': True,
        'concurrent_fragment_downloads': 10
    }

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([video_url])
        print(f"✅ 다운로드 완료: {query}.mp3")
    except Exception as e:
        print(f"❌ 다운로드 실패: {query} → {e}")

# ✅ 곡마다 반복

for i, row in df.iterrows():
    title = str(row['title']).strip()
    singer = str(row['singer']).strip()
    query = f"{singer} {title}"

    # 공백 제거 안 함! 원래 제목 그대로 저장
    mp3_filename = f"{query}.mp3"
    output_dir = "/content/drive/MyDrive/graduateproject/music_lyric/mp3s"
    mp3_path = os.path.join(output_dir, mp3_filename)

    if os.path.exists(mp3_path):
        print(f"⏭️ 이미 존재함, 건너뜀: {mp3_filename}")
        continue

    download_mp3_from_youtube(query, output_dir=output_dir)



⏭️ 이미 존재함, 건너뜀: 아이유 Love wins all.mp3
⏭️ 이미 존재함, 건너뜀: 아이유 Blueming.mp3
⏭️ 이미 존재함, 건너뜀: 아이유 Celebrity.mp3
⏭️ 이미 존재함, 건너뜀: 아이유 밤편지.mp3
⏭️ 이미 존재함, 건너뜀: 아이유 나의 옛날이야기.mp3
